# Blog post analysis

In [1]:
# import Bach
from bach_open_taxonomy import ObjectivFrame

In [2]:
# connect to SQL db
of = ObjectivFrame.from_objectiv_data(db_url='postgresql://postgres:@localhost:5432/objectiv',
                                      table_name='data',
                                      time_aggregation='YYYY-MM-DD',
                                      start_date='2022-02-02')

In [3]:
# adding specific contexts to the data
of['application'] = of.global_contexts.gc.get_from_context_with_type_series(type='ApplicationContext', key='id')
of['root_location'] = of.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
of['feature_nice_name'] = of.location_stack.ls.nice_name

## We had 400 users on the site following the post

In [4]:
# model hub: unique users, total month
users = of.model_hub.aggregate.unique_users(time_aggregation='YYYY-MM')
users.sort_index(ascending=False).head(10)

moment
2022-02    400
Name: unique_users, dtype: int64

In [5]:
# model hub: unique users, daily
users = of.model_hub.aggregate.unique_users()
users.sort_index(ascending=False).head(10)

moment
2022-02-07     10
2022-02-06     20
2022-02-05     33
2022-02-04     53
2022-02-03    100
2022-02-02    239
Name: unique_users, dtype: int64

## They spent on average 4.24 mins on site

In [6]:
# model hub: duration
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM')
duration.sort_index(ascending=False).head(10)

moment
2022-02   0 days 00:04:24.078493
Name: session_duration, dtype: timedelta64[ns]

## Most users on blog, then home, then docs

In [7]:
# users by RootLocation
users_root = of.groupby(['root_location']).agg({'user_id':'nunique'})
users_root.sort_values('user_id_nunique', ascending=False).head()

,user_id_nunique
root_location,
blog,228
home,174
docs,132
about,56
jobs,49


## Users spent 6.24 mins(!) on home, 6.01 in docs 

In [8]:
of[(of.root_location == 'home')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:06:24.071506
Name: session_duration, dtype: timedelta64[ns]

In [10]:
of[(of.root_location == 'docs')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:06:01.047539
Name: session_duration, dtype: timedelta64[ns]

In [11]:
of[(of.root_location == 'blog')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:02:29.157431
Name: session_duration, dtype: timedelta64[ns]

## Video Bach did well in blog and on home (adding in blog was good)

In [12]:
# select PressEvent and MediaStart event, to focus on user actions
users_feature = of[(of.event_type == 'PressEvent') |
                   (of.event_type == 'MediaStartEvent')]

# users by feature
users_feature = users_feature.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
users_feature.sort_values('user_id_nunique', ascending=False).head()

,,user_id_nunique
feature_nice_name,event_type,
Media Player: objectiv-in-2-minutes located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc,MediaStartEvent,92
Media Player: 2-minute-video located at Root Location: home,MediaStartEvent,50
Link: about-us located at Root Location: home => Navigation: navbar-top,PressEvent,32
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,PressEvent,30
Link: logo located at Root Location: blog => Navigation: navbar-top,PressEvent,28


## Docs: Modeling, Tracking, Quickstart even

In [14]:
# users by feature
users_feature.sort_values('user_id_nunique', ascending=False).head(15)

,,user_id_nunique
feature_nice_name,event_type,
Media Player: objectiv-in-2-minutes located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc,MediaStartEvent,92
Media Player: 2-minute-video located at Root Location: home,MediaStartEvent,50
Link: about-us located at Root Location: home => Navigation: navbar-top,PressEvent,32
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,PressEvent,30
Link: logo located at Root Location: blog => Navigation: navbar-top,PressEvent,28
Pressable: hamburger located at Root Location: blog => Navigation: navbar-top,PressEvent,27
Link: quickstart-guide located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc => Content: blog-post-try-objectiv,PressEvent,16
Link: docs located at Root Location: home => Navigation: navbar-top,PressEvent,14
Link: Modeling located at Root Location: docs => Navigation: navbar-top,PressEvent,14


## GitHub links small numbers, on blog & home did better than top nav

In [15]:
# select github LinkContext only
of['link_id'] = of.location_stack.ls.get_from_context_with_type_series(type='LinkContext', key='id')
github_users = of[(of.link_id == 'objectiv-on-github')|
                  (of.link_id == 'github')]

# github users by feature
github_users = github_users.groupby(['feature_nice_name']).agg({'user_id':'nunique'})
github_users.sort_values('user_id_nunique', ascending=False).head()

,user_id_nunique
feature_nice_name,
Link: objectiv-on-github located at Root Location: home => Content: the-stack,11
Link: objectiv-on-github located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc => Content: blog-post-try-objectiv,10
Link: github located at Root Location: home => Navigation: navbar-top,5
Link: github located at Root Location: blog => Navigation: navbar-top => Overlay: hamburger-menu,2
Link: github located at Root Location: home => Navigation: navbar-top => Overlay: hamburger-menu,2


## What did users do most on home?

In [16]:
# select homepage RootLocation only
home_users = of[(of.root_location == 'home')]

# select PressEvent and MediaStart event, to focus on user actions
home_users = home_users[(home_users.event_type == 'PressEvent') |
                   (home_users.event_type == 'MediaStartEvent')]

# home users by feature
home_users = home_users.groupby(['feature_nice_name']).agg({'user_id':'nunique'})
home_users.sort_values('user_id_nunique', ascending=False).head(10)

,user_id_nunique
feature_nice_name,
Media Player: 2-minute-video located at Root Location: home,50
Link: about-us located at Root Location: home => Navigation: navbar-top,32
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,30
Link: docs located at Root Location: home => Navigation: navbar-top,14
Link: logo located at Root Location: home => Navigation: navbar-top,13
Link: blog located at Root Location: home => Navigation: navbar-top,12
Link: objectiv-on-github located at Root Location: home => Content: the-stack,11
Link: about-us located at Root Location: home => Navigation: navbar-top => Overlay: hamburger-menu,11
Link: faq located at Root Location: home => Navigation: navbar-top,7
